In [1]:
import pandas as pd
import os
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn import metrics

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [4]:
os.chdir('/Users/alexanderwendelborn/Documents/')

In [5]:
#buffalo = pd.read_csv('buffalo_weather_data.csv')

In [6]:
#houston = pd.read_csv('houston_weather_data.csv')

In [7]:
#newyork = pd.read_csv('laguardia_weather_data.csv')

In [8]:
#miami = pd.read_csv('miami_weather_data.csv')

In [9]:
#minny = pd.read_csv('minneapolis_weather_data.csv')

In [10]:
#buffalo.name='buffalo'

In [11]:
#houston.name='houston'

In [12]:
#newyork.name='newyork'

In [13]:
#miami.name='miami'

In [14]:
#minny.name='minneapolis'

In [15]:
#df = [buffalo,houston,newyork,miami,minny]

In [16]:
#data = pd.DataFrame()

In [17]:
#for file in df:
#    file['AvTemp_'+file.name]=(file.TMAX/10+file.TMIN/10)/2
#    file.rename(columns={'PRCP':'PRCP_'+file.name},inplace=True)
#    file.drop(columns=['TMAX','TMIN'],inplace=True)
#    if file.name=='buffalo':
#        data=file
#    else:
#        data=pd.merge(data,file,on='date',how='outer')

In [73]:
data=pd.read_csv('Alex_Weather_Data.csv')
#read in data

In [74]:
data.date=pd.to_datetime(data.date)

In [76]:
data.drop(columns='Unnamed: 0',inplace=True)

In [79]:
data.AvTemp_buffalo=data.AvTemp_buffalo.rolling(7).mean()
data.AvTemp_houston=data.AvTemp_houston.rolling(7).mean()
data.AvTemp_newyork=data.AvTemp_newyork.rolling(7).mean()
data.AvTemp_miami=data.AvTemp_miami.rolling(7).mean()
data.AvTemp_minneapolis=data.AvTemp_minneapolis.rolling(7).mean()
#get weekly temperatures

In [80]:
data.loc[data.PRCP_buffalo>0, 'PRCP_buffalo']=1
#turn precipication into a dummy variable

In [81]:
data.loc[data.PRCP_houston>0, 'PRCP_houston']=1
data.loc[data.PRCP_newyork>0, 'PRCP_newyork']=1
data.loc[data.PRCP_miami>0, 'PRCP_miami']=1
data.loc[data.PRCP_minneapolis>0, 'PRCP_minneapolis']=1

In [82]:
data.PRCP_buffalo=data.PRCP_buffalo.rolling(7).sum()
data.PRCP_houston=data.PRCP_houston.rolling(7).sum()
data.PRCP_newyork=data.PRCP_newyork.rolling(7).sum()
data.PRCP_miami=data.PRCP_miami.rolling(7).sum()
data.PRCP_minneapolis=data.PRCP_minneapolis.rolling(7).sum()
#rolling sums

In [83]:
data=pd.DataFrame(data.iloc[6:,:])

In [84]:
data['Month']=data.date.dt.month
data['Day']=data.date.dt.day

In [85]:
data.drop(list(data[(data.Month==2)&(data.Day==29)].index),inplace=True)

In [86]:
data.drop(list(data[(data.Month==8)&(data.Day==30)].index),inplace=True)
#drop these two days for ease of finding yearly averages

In [87]:
data.reset_index(drop=True,inplace=True)

In [88]:
data=data.iloc[list(range(0,5215,7))]

In [89]:
data, val = train_test_split(data,train_size=.8,random_state=6548)

In [90]:
#standardize data

In [91]:
annualaverages = data[['AvTemp_buffalo','AvTemp_houston','AvTemp_newyork','AvTemp_miami','AvTemp_minneapolis','PRCP_buffalo',
                      'PRCP_houston','PRCP_newyork','PRCP_miami','PRCP_minneapolis']].groupby([data.Day,data.Month]).mean().reset_index()

In [92]:
annualstd = data[['AvTemp_buffalo','AvTemp_houston','AvTemp_newyork','AvTemp_miami','AvTemp_minneapolis','PRCP_buffalo',
                      'PRCP_houston','PRCP_newyork','PRCP_miami','PRCP_minneapolis']].groupby([data.Day,data.Month]).std().reset_index()

In [93]:
annualaverages.rename(columns={'AvTemp_buffalo':'AvEveryYear_buffalo','AvTemp_houston':'AvEveryYear_houston',
                               'AvTemp_newyork':'AvEveryYear_newyork','AvTemp_miami':'AvEveryYear_miami',
                              'AvTemp_minneapolis':'AvEveryYear_minneapolis','PRCP_buffalo':'AvPRCP_buffalo',
                                'PRCP_houston':'AvPRCP_houston','PRCP_newyork':'AvPRCP_newyork',
                               'PRCP_miami':'AvPRCP_miami','PRCP_minneapolis':'AvPRCP_minneapolis'},inplace=True)

In [94]:
annualstd.rename(columns={'AvTemp_buffalo':'StdEveryYear_buffalo','AvTemp_houston':'StdEveryYear_houston',
                               'AvTemp_newyork':'StdEveryYear_newyork','AvTemp_miami':'StdEveryYear_miami',
                              'AvTemp_minneapolis':'StdEveryYear_minneapolis','PRCP_buffalo':'StdPRCP_buffalo',
                              'PRCP_houston':'StdPRCP_houston','PRCP_newyork':'StdPRCP_newyork',
                              'PRCP_miami':'StdPRCP_miami','PRCP_minneapolis':'StdPRCP_minneapolis'},inplace=True)

In [95]:
data = pd.merge(data,annualaverages,how='left',on=['Month','Day'])
val = pd.merge(val,annualaverages,how='left',on=['Month','Day'])

In [96]:
data = pd.merge(data,annualstd,how='left',on=['Month','Day'])
val = pd.merge(val,annualstd,how='left',on=['Month','Day'])

In [97]:
savet=data
#save for unstandardizing at the end

In [98]:
savev=val

In [99]:
metrics.mean_squared_error(savet.AvTemp_newyork,savet.AvEveryYear_newyork,squared=False)
#training MSE using average temp for that week every year from training set

2.555071463528904

In [100]:
metrics.mean_squared_error(savev.AvTemp_newyork,savev.AvEveryYear_newyork,squared=False)
#validation MSE using average temp for that week every year from training set

2.6800741326270403

In [101]:
data['VarTemp_buffalo']=(data.AvTemp_buffalo-data.AvEveryYear_buffalo)/data.StdEveryYear_buffalo
data['VarPRCP_buffalo']=(data.PRCP_buffalo-data.AvPRCP_buffalo)/data.StdPRCP_buffalo
data['VarTemp_houston']=(data.AvTemp_houston-data.AvEveryYear_houston)/data.StdEveryYear_houston
data['VarPRCP_houston']=(data.PRCP_houston-data.AvPRCP_houston)/data.StdPRCP_houston
data['VarTemp_newyork']=(data.AvTemp_newyork-data.AvEveryYear_newyork)/data.StdEveryYear_newyork
data['VarPRCP_newyork']=(data.PRCP_newyork-data.AvPRCP_newyork)/data.StdPRCP_newyork
data['VarTemp_miami']=(data.AvTemp_miami-data.AvEveryYear_miami)/data.StdEveryYear_miami
data['VarPRCP_miami']=(data.PRCP_miami-data.AvPRCP_miami)/data.StdPRCP_miami
data['VarTemp_minneapolis']=(data.AvTemp_minneapolis-data.AvEveryYear_minneapolis)/data.StdEveryYear_minneapolis
data['VarPRCP_minneapolis']=(data.PRCP_minneapolis-data.AvPRCP_minneapolis)/data.StdPRCP_minneapolis

val['VarTemp_buffalo']=(val.AvTemp_buffalo-val.AvEveryYear_buffalo)/val.StdEveryYear_buffalo
val['VarPRCP_buffalo']=(val.PRCP_buffalo-val.AvPRCP_buffalo)/val.StdPRCP_buffalo
val['VarTemp_houston']=(val.AvTemp_houston-val.AvEveryYear_houston)/val.StdEveryYear_houston
val['VarPRCP_houston']=(val.PRCP_houston-val.AvPRCP_houston)/val.StdPRCP_houston
val['VarTemp_newyork']=(val.AvTemp_newyork-val.AvEveryYear_newyork)/val.StdEveryYear_newyork
val['VarPRCP_newyork']=(val.PRCP_newyork-val.AvPRCP_newyork)/val.StdPRCP_newyork
val['VarTemp_miami']=(val.AvTemp_miami-val.AvEveryYear_miami)/val.StdEveryYear_miami
val['VarPRCP_miami']=(val.PRCP_miami-val.AvPRCP_miami)/val.StdPRCP_miami
val['VarTemp_minneapolis']=(val.AvTemp_minneapolis-val.AvEveryYear_minneapolis)/val.StdEveryYear_minneapolis
val['VarPRCP_minneapolis']=(val.PRCP_minneapolis-val.AvPRCP_minneapolis)/val.StdPRCP_minneapolis
#standardize

In [102]:
data=data[['date','VarTemp_buffalo','VarPRCP_buffalo','VarTemp_houston','VarPRCP_houston','VarTemp_newyork','VarPRCP_newyork','VarTemp_miami','VarPRCP_miami','VarTemp_minneapolis','VarPRCP_minneapolis']]
val=val[['date','VarTemp_buffalo','VarPRCP_buffalo','VarTemp_houston','VarPRCP_houston','VarTemp_newyork','VarPRCP_newyork','VarTemp_miami','VarPRCP_miami','VarTemp_minneapolis','VarPRCP_minneapolis']]

In [103]:
data.fillna(0,inplace=True)
val.fillna(0,inplace=True)

/Users/alexanderwendelborn/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [104]:
data['z']='train'
val['z']='val'
# for filtering after adding lags

<ipython-input-104-cea01d176919>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['z']='train'
<ipython-input-104-cea01d176919>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val['z']='val'


In [105]:
data=data.append(val)

In [106]:
data.set_index('date',inplace=True)

In [107]:
data.sort_values('date',ascending=True,inplace=True)

In [108]:
loop = list(range(1,53,1))

In [109]:
output=data

In [110]:
for lag in loop:
    lagged = data.shift(periods=lag)
    lagged.rename(columns={'VarTemp_buffalo':'VarTemp_buffalo'+'_lag'+str(lag).zfill(2),'VarPRCP_buffalo':'VarPRCP_buffalo'+'_lag'+str(lag).zfill(2),'VarTemp_houston':'VarTemp_houston'+'_lag'+str(lag).zfill(2),'VarPRCP_houston':'VarPRCP_houston'+'_lag'+str(lag).zfill(2),'VarTemp_newyork':'VarTemp_newyork'+'_lag'+str(lag).zfill(2),'VarPRCP_newyork':'VarPRCP_newyork'+'_lag'+str(lag).zfill(2),'VarTemp_miami':'VarTemp_miami'+'_lag'+str(lag).zfill(2),'VarPRCP_miami':'VarPRCP_miami'+'_lag'+str(lag).zfill(2),'VarTemp_minneapolis':'VarTemp_minneapolis'+'_lag'+str(lag).zfill(2),'VarPRCP_minneapolis':'VarPRCP_minneapolis'+'_lag'+str(lag).zfill(2)},inplace=True)
    lagged.drop(columns=['z'],inplace=True)
    output = pd.merge(output,lagged, left_index=True, right_index=True)
    #create lagged columns

In [111]:
needsfeatures=output.iloc[52:,:]

In [112]:
needsfeatures.sort_index(axis=1,inplace=True)

<ipython-input-112-ce69d739519d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needsfeatures.sort_index(axis=1,inplace=True)


In [113]:
ys=needsfeatures.VarTemp_newyork

In [114]:
needsfeatures.drop(columns=['VarPRCP_newyork','VarTemp_newyork','VarTemp_buffalo','VarPRCP_buffalo','VarTemp_houston','VarPRCP_houston','VarTemp_miami','VarPRCP_miami','VarTemp_minneapolis','VarPRCP_minneapolis'],inplace=True)

/Users/alexanderwendelborn/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [115]:
# add averages for lags far in the past

In [116]:
needsfeatures['buffaloPRCP_lag5:lag8']=needsfeatures.iloc[:,4:8].mean(axis=1)
needsfeatures['houstonPRCP_lag5:lag8']=needsfeatures.iloc[:,56:60].mean(axis=1)
needsfeatures['miamiPRCP_lag5:lag8']=needsfeatures.iloc[:,108:112].mean(axis=1)
needsfeatures['minneapolisPRCP_lag5:lag8']=needsfeatures.iloc[:,160:164].mean(axis=1)
needsfeatures['newyorkPRCP_lag5:lag8']=needsfeatures.iloc[:,212:216].mean(axis=1)
needsfeatures['buffaloTemp_lag5:lag8']=needsfeatures.iloc[:,264:268].mean(axis=1)
needsfeatures['houstonTemp_lag5:lag8']=needsfeatures.iloc[:,316:320].mean(axis=1)
needsfeatures['miamiTemp_lag5:lag8']=needsfeatures.iloc[:,368:372].mean(axis=1)
needsfeatures['minneapolisTemp_lag5:lag8']=needsfeatures.iloc[:,420:424].mean(axis=1)
needsfeatures['newyorkTemp_lag5:lag8']=needsfeatures.iloc[:,472:476].mean(axis=1)

<ipython-input-116-b30e3e036287>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needsfeatures['buffaloPRCP_lag5:lag8']=needsfeatures.iloc[:,4:8].mean(axis=1)
<ipython-input-116-b30e3e036287>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needsfeatures['houstonPRCP_lag5:lag8']=needsfeatures.iloc[:,56:60].mean(axis=1)
<ipython-input-116-b30e3e036287>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [117]:
needsfeatures['buffaloPRCP_lag9:lag16']=needsfeatures.iloc[:,8:16].mean(axis=1)
needsfeatures['houstonPRCP_lag9:lag16']=needsfeatures.iloc[:,60:68].mean(axis=1)
needsfeatures['miamiPRCP_lag9:lag16']=needsfeatures.iloc[:,112:120].mean(axis=1)
needsfeatures['minneapolisPRCP_lag9:lag16']=needsfeatures.iloc[:,164:172].mean(axis=1)
needsfeatures['newyorkPRCP_lag9:lag16']=needsfeatures.iloc[:,216:224].mean(axis=1)
needsfeatures['buffaloTemp_lag9:lag16']=needsfeatures.iloc[:,268:276].mean(axis=1)
needsfeatures['houstonTemp_lag9:lag16']=needsfeatures.iloc[:,320:328].mean(axis=1)
needsfeatures['miamiTemp_lag9:lag16']=needsfeatures.iloc[:,372:380].mean(axis=1)
needsfeatures['minneapolisTemp_lag9:lag16']=needsfeatures.iloc[:,424:432].mean(axis=1)
needsfeatures['newyorkTemp_lag9:lag16']=needsfeatures.iloc[:,476:484].mean(axis=1)

<ipython-input-117-d8ff58ba5332>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needsfeatures['buffaloPRCP_lag9:lag16']=needsfeatures.iloc[:,8:16].mean(axis=1)
<ipython-input-117-d8ff58ba5332>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needsfeatures['houstonPRCP_lag9:lag16']=needsfeatures.iloc[:,60:68].mean(axis=1)
<ipython-input-117-d8ff58ba5332>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

Se

In [118]:
needsfeatures['buffaloPRCP_lag17:lag30']=needsfeatures.iloc[:,16:30].mean(axis=1)
needsfeatures['houstonPRCP_lag17:lag30']=needsfeatures.iloc[:,68:82].mean(axis=1)
needsfeatures['miamiPRCP_lag17:lag30']=needsfeatures.iloc[:,120:134].mean(axis=1)
needsfeatures['minneapolisPRCP_lag17:lag30']=needsfeatures.iloc[:,172:186].mean(axis=1)
needsfeatures['newyorkPRCP_lag17:lag30']=needsfeatures.iloc[:,224:238].mean(axis=1)
needsfeatures['buffaloTemp_lag17:lag30']=needsfeatures.iloc[:,276:290].mean(axis=1)
needsfeatures['houstonTemp_lag17:lag30']=needsfeatures.iloc[:,328:342].mean(axis=1)
needsfeatures['miamiTemp_lag17:lag30']=needsfeatures.iloc[:,380:394].mean(axis=1)
needsfeatures['minneapolisTemp_lag17:lag30']=needsfeatures.iloc[:,432:446].mean(axis=1)
needsfeatures['newyorkTemp_lag17:lag30']=needsfeatures.iloc[:,484:498].mean(axis=1)

<ipython-input-118-de7f8dda15e4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needsfeatures['buffaloPRCP_lag17:lag30']=needsfeatures.iloc[:,16:30].mean(axis=1)
<ipython-input-118-de7f8dda15e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needsfeatures['houstonPRCP_lag17:lag30']=needsfeatures.iloc[:,68:82].mean(axis=1)
<ipython-input-118-de7f8dda15e4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead


In [119]:
needsfeatures['buffaloPRCP_lag31:lag52']=needsfeatures.iloc[:,30:52].mean(axis=1)
needsfeatures['houstonPRCP_lag31:lag52']=needsfeatures.iloc[:,82:104].mean(axis=1)
needsfeatures['miamiPRCP_lag31:lag52']=needsfeatures.iloc[:,134:156].mean(axis=1)
needsfeatures['minneapolisPRCP_lag31:lag52']=needsfeatures.iloc[:,186:208].mean(axis=1)
needsfeatures['newyorkPRCP_lag31:lag52']=needsfeatures.iloc[:,238:260].mean(axis=1)
needsfeatures['buffaloTemp_lag31:lag52']=needsfeatures.iloc[:,290:312].mean(axis=1)
needsfeatures['houstonTemp_lag31:lag52']=needsfeatures.iloc[:,342:364].mean(axis=1)
needsfeatures['miamiTemp_lag31:lag52']=needsfeatures.iloc[:,394:416].mean(axis=1)
needsfeatures['minneapolisTemp_lag31:lag52']=needsfeatures.iloc[:,446:468].mean(axis=1)
needsfeatures['newyorkemp_lag31:lag52']=needsfeatures.iloc[:,498:520].mean(axis=1)

<ipython-input-119-3c0ca12e11ec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needsfeatures['buffaloPRCP_lag31:lag52']=needsfeatures.iloc[:,30:52].mean(axis=1)
<ipython-input-119-3c0ca12e11ec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needsfeatures['houstonPRCP_lag31:lag52']=needsfeatures.iloc[:,82:104].mean(axis=1)
<ipython-input-119-3c0ca12e11ec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [120]:
one=needsfeatures.iloc[:,520:]

In [121]:
two=needsfeatures.iloc[:,0:4]
three=needsfeatures.iloc[:,52:56]
four=needsfeatures.iloc[:,104:108]
five=needsfeatures.iloc[:,156:160]
six=needsfeatures.iloc[:,208:212]
seven=needsfeatures.iloc[:,260:264]
eight=needsfeatures.iloc[:,312:316]
nine=needsfeatures.iloc[:,364:368]
ten=needsfeatures.iloc[:,416:420]
eleven=needsfeatures.iloc[:,468:472]

In [122]:
aftereng=pd.concat([one,two,three,four,five,six,seven,eight,nine,ten,eleven,ys],axis=1)
#combine engineered features

In [123]:
train=aftereng[aftereng.z=='train']

In [124]:
val=aftereng[aftereng.z=='val']

In [125]:
train.drop(columns='z',inplace=True)
val.drop(columns='z',inplace=True)

In [126]:
#train.reset_index(drop=True,inplace=True)
#val.reset_index(drop=True,inplace=True)

In [127]:
trainx=train.iloc[:,:80]
valx=val.iloc[:,:80]

In [128]:
trainy=train.VarTemp_newyork
valy=val.VarTemp_newyork

In [130]:
#train model

In [131]:
parameters = {'max_depth': range(1, 10), 'n_estimators': [50, 100], 'subsample': [1], 'colsample_bytree': [1], 'colsample_bynode': [1], 'gamma': [0, .5,2,5, 10, 15, 20], 'learning_rate': [.01,.03,.05,.07,.1,0.15, 0.2, 0.25, 0.3], 'alpha': [0.1, 0.25, 0.5, 0.75, 1]}

In [132]:
xgmodel = RandomizedSearchCV(xgb.XGBRegressor(eval_metric ='rmse'), parameters, n_jobs=-1, 
                             scoring ='neg_mean_squared_error',n_iter = 500, random_state = 5)

In [133]:
xgmodel.fit(trainx, trainy)

RandomizedSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          eval_metric='rmse', gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_es...
                                          validate_parameters=None,
                                          verbosity=None),
                   n_iter=500, n_jobs=-1,

In [134]:
valpredict=xgmodel.predict(valx)
trainpredict=xgmodel.predict(trainx)

In [135]:
metrics.mean_squared_error(trainy,trainpredict,squared=False)

0.7488574075183279

In [136]:
metrics.mean_squared_error(valy,valpredict,squared=False)

1.0144099457523164

In [137]:
valy.std()

1.1057923093826694

In [138]:
trainy.std()

0.9544313994851964

In [139]:
# unstandardize

In [140]:
trainy=trainy.reset_index()
valy=valy.reset_index()

In [141]:
trainy['prediction']=list(trainpredict)
valy['prediction']=list(valpredict)

In [142]:
trainy=pd.merge(savet,trainy,on='date')
valy=pd.merge(savev,valy,on='date')

In [143]:
trainy['Prediction in C']=trainy.prediction*trainy.StdEveryYear_newyork+trainy.AvEveryYear_newyork
valy['Prediction in C']=valy.prediction*valy.StdEveryYear_newyork+valy.AvEveryYear_newyork

In [144]:
metrics.mean_squared_error(trainy.AvTemp_newyork,trainy['Prediction in C'],squared=False)

1.991402038631688

In [145]:
metrics.mean_squared_error(valy.AvTemp_newyork,valy['Prediction in C'],squared=False)

2.359861137817013

In [146]:
# feature importances

In [147]:
importance = pd.DataFrame()

In [148]:
importance['Variable']=trainx.columns

In [149]:
importance['Importance']=xgmodel.best_estimator_.feature_importances_

In [150]:
importance.sort_values('Importance', ascending=False,inplace=True)

In [151]:
importance.reset_index(inplace=True,drop=True)

In [152]:
importance.iloc[:30,:]

,Variable,Importance
0,VarTemp_minneapolis_lag01,0.06955
1,VarTemp_buffalo_lag01,0.05249
2,newyorkTemp_lag17:lag30,0.04091
3,minneapolisTemp_lag17:lag30,0.03770
4,VarTemp_newyork_lag04,0.03182
5,newyorkTemp_lag9:lag16,0.03158
6,buffaloPRCP_lag9:lag16,0.03025
7,VarTemp_newyork_lag02,0.02949
8,miamiTemp_lag31:lag52,0.02780
9,VarPRCP_buffalo_lag02,0.02733
